#세팅

In [1]:
!pip install selenium
!apt-get update

# (최초 1회)
!apt install chromium-chromedriver
!cp /usr/lib/chromium-browser/chromedriver '/content/drive/MyDrive/Colab Notebooks' #
!pip install chromedriver-autoinstaller

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 48.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 476.0/476.0 kB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Ign:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:11 http://security.ubuntu.

In [2]:
!python --version

import selenium
print(selenium.__version__)

from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import sys
from selenium.webdriver.common.keys import Keys
import urllib.request
import os
from urllib.request import urlretrieve

import time
import pandas as pd
import numpy as np
import chromedriver_autoinstaller  # setup chrome options

chrome_path = "/content/drive/MyDrive/Colab Notebooks/chromedriver"

sys.path.insert(0,chrome_path)
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') # ensure GUI is off
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')  # set path to chromedriver as per your configuration
chrome_options.add_argument('lang=ko_KR') # 한국어
chrome_options.add_argument('user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36')

chromedriver_autoinstaller.install()  # set the target URL

Python 3.10.12
4.31.1


#상품정보를 데이터프레임으로 가져오기

In [6]:
# 코드 입력
import re
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import numpy as np

def scrape_course_info(url):
    # Selenium driver 로드
    driver = webdriver.Chrome(options=chrome_options)

    # 링크 전달
    driver.get(url)

    #CSS_SELECTOR를 활용해서 div 태그를 가져오기
    div = driver.find_element(By.CSS_SELECTOR, "div.prd_info")

    # 정보 저장을 위한 딕셔너리 초기화
    product_info = {}

    # 제목 가져오기
    title = div.find_element(By.CSS_SELECTOR, "p.prd_name")
    product_info['title'] = title.text

    # 제조사 가져오기
    manufacturer = div.find_element(By.CSS_SELECTOR, "p.prd_brand")
    product_info['manufacturer'] = manufacturer.text

    #가격 가져오기 정가/할인가

    div_price = driver.find_element(By.CSS_SELECTOR, "div.price")
    try:  # 할인이 있는 경우
        price = div_price.find_element(By.CSS_SELECTOR, "span.price-2")
        price_text = price.text.split('원')[0].replace(',', '')
        price_int = int(price_text)

    except:
        try: # 할인이 없는 경우
            price = div_price.find_element(By.CSS_SELECTOR, "span.price-1")
            price_text = price.text.split('원')[0].replace(',', '')
            price_int = int(price_text)

        except:
         price_int = np.nan
    product_info['price'] = price_int

    # 리뷰 정보
    # 리뷰 등급 요소 찾기
    def convert_review_grade(grade):
        if grade == "최고":
            return 5
        elif grade == "좋음":
            return 4
        elif grade == "보통":
            return 3
        elif grade == "별로":
            return 2
        elif grade == "나쁨":
            return 1
        else:
            return None

    # 리뷰 등급 텍스트 가져오기
    # 등급이 포함된 요소를 찾기 위해 CSS 선택자 사용
    review_elements = driver.find_elements(By.CSS_SELECTOR, '.grade_img .grade')
    # review_elements = driver.find_elements(By.CSS_SELECTOR, 'div.grade_img > span.grade > em') # span과 em은 자식관계가 아닌 형제관계로 '>' 사용은 부적절

    # 리뷰 정보
    # '리뷰' 버튼을 클릭하기 위해 WebDriverWait 사용
    review_button = WebDriverWait(driver, 10).until(
      EC.element_to_be_clickable((By.XPATH, '//*[@id="reviewInfo"]/a'))
      )
    review_button.click()

    # 리뷰가 로드될 때까지 잠시 대기
    time.sleep(5)  # 페이지가 로드될 시간을 충분히 제공
    # 리뷰 등급 요소 찾기
    # 리뷰 등급 텍스트 가져오기
    # 등급이 포함된 요소를 찾기 위해 CSS 선택자 사용
    review_elements = driver.find_elements(By.CSS_SELECTOR, '.grade_img .grade')
    # review_elements = driver.find_elements(By.CSS_SELECTOR, 'div.grade_img > span.grade > em') # span과 em은 자식관계가 아닌 형제관계로 '>' 사용은 부적절

    for element in review_elements:
        grade_class = element.get_attribute('class')
        grade = element.find_element(By.XPATH, './following-sibling::em').text.strip()
        product_info['등급클래스'] = grade_class
    # 리뷰 등급 텍스트를 숫자로 변환
    review_grade = convert_review_grade(grade)


    # 리뷰 수 & 리뷰 별점
    review_count = driver.find_element(By.CSS_SELECTOR, "p.total > em").text
    review_average = driver.find_element(By.CSS_SELECTOR, "p.num > strong").text

    product_info['리뷰수'] = review_count ; product_info['평점'] = review_average

    # 리뷰 점수별 퍼센트 크롤링
    product_info['세부평점'] = ""

    li_elements = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'div.graph_area ul.graph_list li'))
    )

    ratings = []

    for li in li_elements:
        # 각 li 요소 내의 span.per와 span.txt 선택
        per_text = li.find_element(By.CSS_SELECTOR, 'span.per').text.strip()
        txt_text = li.find_element(By.CSS_SELECTOR, 'span.txt').text.strip()

        ratings.append(f"{txt_text}: {per_text}")

    # 리스트를 쉼표로 구분된 문자열로 합치기
    product_info['세부평점'] = ", ".join(ratings)

    # 피부타입, 피부고민, 자극도 항목별 퍼센트 크롤링
    poll_elements = WebDriverWait(driver, 10).until(
        EC.presence_of_all_elements_located((By.CSS_SELECTOR, 'dl.poll_type2.type3 ul.list li'))
        )
    results = []
    for poll in poll_elements:
        # 각 li 요소 내의 span.per와 span.txt 선택
        poll_per_text = poll.find_element(By.CSS_SELECTOR, 'em.per').text.strip()
        poll_txt_text = poll.find_element(By.CSS_SELECTOR, 'span.txt').text.strip()

        combined_text = f"{poll_txt_text}, {poll_per_text}"
        results.append(combined_text)
    # 결과를 3개의 리스트로 나누기
    types = results[:3]
    concerns = results[3:6]
    irritation = results[6:9]

    product_info['피부타입'] = types
    product_info['피부고민'] = concerns
    product_info['자극도'] = irritation

    # driver 종료
    driver.quit()

    return product_info

product_info = scrape_course_info("https://www.oliveyoung.co.kr/store/goods/getGoodsDetail.do?goodsNo=A000000202894&t_page=%ED%86%B5%ED%95%A9%EA%B2%80%EC%83%89%EA%B2%B0%EA%B3%BC%ED%8E%98%EC%9D%B4%EC%A7%80&t_click=%EA%B2%80%EC%83%89%EC%83%81%ED%92%88%EC%83%81%EC%84%B8&t_search_name=%EC%88%98%EB%B6%84%ED%81%AC%EB%A6%BC&t_number=1&dispCatNo=1000001000100150001&trackingCd=Result_1")
print(product_info)

{'title': '[8월올영픽/수분천재크림] 에스네이처 아쿠아 스쿠알란 수분크림 60ml 더블 기획 (60ml+60ml+카밍패드 2매)', 'manufacturer': '에스네이처', 'price': 21300, '등급클래스': 'grade grade5', '리뷰수': '24,184', '평점': '4.8', '세부평점': '5점: 84%, 4점: 11%, 3점: 3%, 2점: 1%, 1점: 1%', '피부타입': ['건성에 좋아요, 37%', '복합성에 좋아요, 52%', '지성에 좋아요, 11%'], '피부고민': ['보습에 좋아요, 76%', '진정에 좋아요, 24%', '주름/미백에 좋아요, 1%'], '자극도': ['자극없이 순해요, 76%', '보통이에요, 24%', '자극이 느껴져요, 1%']}


#링크로 5페이지 까지 가져오기

In [10]:
import re
import pandas as pd
from selenium import webdriver
from tqdm import tqdm

# Selenium driver 로드
driver = webdriver.Chrome(options=chrome_options)

# 크롤링 결과를 저장할 리스트
course_info_dicts = []

# 기본 URL 설정 (startCount 부분을 제외한 나머지 URL)
base_url = "https://www.oliveyoung.co.kr/store/search/getSearchMain.do?startCount={}&sort=RANK%2FDESC&goods_sort=WEIGHT%2FDESC%2CRANK%2FDESC&collection=ALL&realQuery=%EC%88%98%EB%B6%84%ED%81%AC%EB%A6%BC&reQuery=&viewtype=image&category=&catename=LCTG_ID&catedepth=1&rt=&setMinPrice=&setMaxPrice=&listnum=24&tmp_requery=&tmp_requery2=&categoryDepthValue=1&cateId=&cateId2=&BenefitAll_CHECK=&query=%EC%88%98%EB%B6%84%ED%81%AC%EB%A6%BC&selectCateNm=%EC%A0%84%EC%B2%B4&firstTotalCount=447&typeChk=thum&branChk=&brandTop=&attrChk=&attrTop=&onlyOneBrand=&quickYn=N&cateChk=opened&benefitChk=&attrCheck0=&attrCheck1=&attrCheck2=&attrCheck3=&attrCheck4=&brandChkList=&benefitChkList=&_displayImgUploadUrl=https%3A%2F%2Fimage.oliveyoung.co.kr%2Fuploads%2Fimages%2Fdisplay%2F&recobellMbrNo=null&recobellCuid=8b47cf9f-efd1-48e4-8f83-10ee8a07945b&t_page=&t_click=&t_search_name=&sale_below_price=&sale_over_price=&reChk="

# 총 몇 페이지를 크롤링할지 설정 (예: 5페이지)
total_pages = 5

# 각 페이지에 대해 크롤링 수행
for page in range(total_pages):
    # startCount 값 계산 (24씩 증가)
    start_count = page * 24

    # 현재 페이지의 URL 생성
    url = base_url.format(start_count)

    # 페이지로 이동
    driver.get(url)

    # CSS_SELECTOR를 활용해서 section 태그를 가져오기
    section = driver.find_element(By.CSS_SELECTOR, "div#ajaxList")

    # a 태그들 가져오기
    a_tags = section.find_elements(By.TAG_NAME, "a")

    # href 속성 가져오기
    hrefs = [a.get_attribute("href") for a in a_tags if a.get_attribute("href")]

    # hrefs를 tqdm으로 감싸서 진행률 표시줄을 추가합니다.
    for href in tqdm(hrefs, desc=f"Scraping course info from page {page + 1}"):
        try:
            course_info_dict = scrape_course_info(href)  # 각 링크에서 크롤링 작업 수행
            course_info_dicts.append(course_info_dict)
        except:
            pass


# driver 종료
driver.quit()

# 결과를 DataFrame으로 변환
inflearn_df = pd.DataFrame(course_info_dicts)
inflearn_df

Scraping course info from page 5: 100%|██████████| 48/48 [07:19<00:00,  9.17s/it]


,title,manufacturer,price,등급클래스,리뷰수,평점,세부평점,피부타입,피부고민,자극도
0,[8월올영픽/수분천재크림] 에스네이처 아쿠아 스쿠알란 수분크림 60ml 더블 기획 ...,에스네이처,21300,grade grade5,"24,197",4.8,"5점: 84%, 4점: 11%, 3점: 3%, 2점: 1%, 1점: 1%","[건성에 좋아요, 37%, 복합성에 좋아요, 52%, 지성에 좋아요, 11%]","[보습에 좋아요, 76%, 진정에 좋아요, 24%, 주름/미백에 좋아요, 1%]","[자극없이 순해요, 76%, 보통이에요, 24%, 자극이 느껴져요, 1%]"
1,[1등수분크림/420ml특대용량] 비욘드 엔젤아쿠아 수분진정크림(150mlx2+12...,비욘드,23500,grade grade5,"5,254",4.8,"5점: 84%, 4점: 11%, 3점: 3%, 2점: 1%, 1점: 1%","[건성에 좋아요, 28%, 복합성에 좋아요, 44%, 지성에 좋아요, 28%]","[보습에 좋아요, 64%, 진정에 좋아요, 36%, 주름/미백에 좋아요, 0%]","[자극없이 순해요, 64%, 보통이에요, 36%, 자극이 느껴져요, 0%]"
2,[수분천재크림] 에스네이처 아쿠아 스쿠알란 수분크림 60ml 기획 (60ml+30ml),에스네이처,17900,grade grade5,"13,861",4.8,"5점: 83%, 4점: 12%, 3점: 4%, 2점: 1%, 1점: 1%","[건성에 좋아요, 35%, 복합성에 좋아요, 56%, 지성에 좋아요, 9%]","[보습에 좋아요, 78%, 진정에 좋아요, 20%, 주름/미백에 좋아요, 2%]","[자극없이 순해요, 77%, 보통이에요, 23%, 자극이 느껴져요, 1%]"
3,[수분크림] 에스트라 아토베리어365 하이드로 수딩크림 60ml,에스트라,29700,grade grade5,"9,380",4.8,"5점: 84%, 4점: 12%, 3점: 3%, 2점: 0%, 1점: 1%","[건성에 좋아요, 27%, 복합성에 좋아요, 54%, 지성에 좋아요, 20%]","[보습에 좋아요, 64%, 진정에 좋아요, 36%, 주름/미백에 좋아요, 1%]","[자극없이 순해요, 81%, 보통이에요, 19%, 자극이 느껴져요, 1%]"
4,[수분크림] 에스트라 아토베리어365 하이드로 수딩크림 60ml 기획(+하이드로에센...,에스트라,29700,grade grade5,"9,680",4.8,"5점: 84%, 4점: 12%, 3점: 3%, 2점: 0%, 1점: 1%","[건성에 좋아요, 27%, 복합성에 좋아요, 58%, 지성에 좋아요, 15%]","[보습에 좋아요, 60%, 진정에 좋아요, 40%, 주름/미백에 좋아요, 1%]","[자극없이 순해요, 78%, 보통이에요, 22%, 자극이 느껴져요, 1%]"
...,...,...,...,...,...,...,...,...,...,...
114,[고민시 PICK]몰바니 아티초크 카밍 케어 수딩 크림 100ml,몰바니,27000,grade grade5,384,4.8,"5점: 88%, 4점: 7%, 3점: 4%, 2점: 1%, 1점: 1%","[건성에 좋아요, 28%, 복합성에 좋아요, 55%, 지성에 좋아요, 17%]","[보습에 좋아요, 54%, 진정에 좋아요, 46%, 주름/미백에 좋아요, 2%]","[자극없이 순해요, 78%, 보통이에요, 20%, 자극이 느껴져요, 2%]"
115,[김지훈 Pick] 미프 비타민 톤업 화이트닝 크림 단독 1+1기획(50ml+50ml),미프,26000,grade grade5,312,4.7,"5점: 81%, 4점: 15%, 3점: 3%, 2점: 1%, 1점: 1%","[건성에 좋아요, 13%, 복합성에 좋아요, 74%, 지성에 좋아요, 13%]","[보습에 좋아요, 15%, 진정에 좋아요, 26%, 주름/미백에 좋아요, 59%]","[자극없이 순해요, 50%, 보통이에요, 50%, 자극이 느껴져요, 1%]"
116,무슈제이 베이직 아쿠아 보습 올인원 1+1 기획(200ml+200ml),무슈제이,24900,grade grade5,657,4.8,"5점: 87%, 4점: 10%, 3점: 3%, 2점: 0%, 1점: 0%","[건성에 좋아요, 23%, 복합성에 좋아요, 65%, 지성에 좋아요, 12%]","[보습에 좋아요, 59%, 진정에 좋아요, 38%, 주름/미백에 좋아요, 3%]","[자극없이 순해요, 63%, 보통이에요, 37%, 자극이 느껴져요, 1%]"
117,바이오더마 하이드라비오 젤크림 40ml,바이오더마,24800,grade grade5,927,4.8,"5점: 83%, 4점: 13%, 3점: 3%, 2점: 1%, 1점: 1%","[건성에 좋아요, 32%, 복합성에 좋아요, 54%, 지성에 좋아요, 14%]","[보습에 좋아요, 69%, 진정에 좋아요, 26%, 주름/미백에 좋아요, 5%]","[자극없이 순해요, 79%, 보통이에요, 21%, 자극이 느껴져요, 1%]"


#데이터수를 100개로 설정 후 csv 인코딩

In [11]:
# 아래 19개 데이터 삭제
inflearn_df1 = inflearn_df[:-19]

# 데이터프레임을 CSV 파일로 저장
inflearn_df1.to_csv('top100상품정보_df.csv', index=True, encoding='utf-8-sig')